In [13]:
from sage.all import *
from sage.crypto.sbox import *
from sage.crypto.sboxes import sboxes
import numpy as np
import pickle
import pandas as pd

In [14]:
# Modified from https://github.com/abrari/block-cipher-testing

# Works only on square s-boxes
def sbox_bic(sbox):

    maxCorr = 0
    m = len(sbox)
    for i in range(m):
        ei = 2**i 
        for j in range(m):
            for k in range(m):
                if j != k:
                    avalanche_vec_j = np.zeros(2**m, dtype=float)
                    avalanche_vec_k = np.zeros(2**m, dtype=float)

                    # Each possible input
                    for X in range(2**m):
                        ej = 2**j 
                        ek = 2**k 

                        dei = sbox[X] ^ sbox[X ^ ei]
                        dej = (dei & ej) >> j
                        dek = (dei & ek) >> k 

                        avalanche_vec_j[X] = dej 
                        avalanche_vec_k[X] = dek
                    corr = abs(np.corrcoef(avalanche_vec_j, avalanche_vec_k)[0,1])
                
                    if maxCorr < corr:
                        maxCorr = corr
    return maxCorr

sbox_bic(sboxes["AES"])


0.1341246360271594

In [21]:
# Modified version to get the average SAC of an sbox from:
#       https://github.com/abrari/block-cipher-testing
def avg_sac(sbox):
    m = len(sbox)
    sac_mat = np.array([np.zeros(m, dtype=float) for _ in range(m)])

    for i in range(m):
        ei = 2**i 
        for j in range(m):
            ej = 2**j 
            for X in range(2**m):
                dei = sbox[X] ^ sbox[X ^ ei]
                sac_mat[i][j] += (dei & ej) >> j 
    
    output_len = 2**m
    for i in range(m):
        for j in range(m):
            sac_mat[i][j] /= output_len

    sum = 0 
    for i in range(m):
        sum += np.sum(sac_mat[i])
    avg = sum / (m**2)
    return avg 

avg_sac(sboxes["AES"])

0.5048828125

In [15]:
nl_dict = {}
lp_dict = {}
dp_dict = {}

for sbox in sboxes.values():
    if len(sbox) == 8:
        nl = sbox.nonlinearity()
        lp = sbox.maximal_linear_bias_relative()
        dp = sbox.maximal_difference_probability()
        if nl in nl_dict.keys():
            nl_dict[nl]+=1
        else:
            nl_dict[nl] = 1

        if lp in lp_dict.keys():
            lp_dict[lp] += 1
        else:
            lp_dict[lp] = 1

        if dp in dp_dict.keys():
            dp_dict[dp] += 1
        else:
            dp_dict[dp] = 1

print(nl_dict)
print(lp_dict)
print(dp_dict)

{112: 11, 94: 5, 102: 1, 100: 8, 96: 20, 0: 1, 88: 1, 64: 4, 104: 4, 90: 1, 82: 1, 92: 1}
{0.0625: 11, 0.1328125: 5, 0.1015625: 1, 0.109375: 8, 0.125: 20, 0.5: 1, 0.15625: 1, 0.25: 4, 0.09375: 4, 0.1484375: 1, 0.1796875: 1, 0.140625: 1}
{0.015625: 12, 0.03125: 17, 0.0390625: 11, 0.046875: 6, 0.5: 2, 0.0625: 6, 0.25: 4}


In [16]:
print(len(nl_dict))
print(len(lp_dict))
print(len(dp_dict))

12
12
7


In [17]:
sboxes["GIFT"]

(1, 10, 4, 12, 6, 15, 3, 9, 2, 13, 11, 7, 5, 0, 8, 14)

In [18]:
def sbox_to_arr(sbox):
    return [num for num in sbox]

Adding the 8 sboxes from the paper

In [40]:
def sboxes_8():
    sboxes = {}
    with open("sboxes_8.txt", "r") as f:
        for line in f.readlines():
            values = line.split(", ")
            values[-1] = values[-1][:4]  # Remove trailing \n 
            sboxes[f"S{values[0]}"] = [int(i, 16) for i in values[1:]]
    return sboxes

Pickle the info for 8-bit sbox

In [42]:
pickle_filename = "sboxes_info.pkl"
df_dict = {}

def create_row(sbox):
    row = {}
    row["box"] = [num for num in sbox] 
    row["inverse"] = [num for num in sbox.inverse()] if sbox.is_permutation() else []
    row["nonlinearity"] = int(sbox.nonlinearity())
    row["linear_probability"] = float(sbox.maximal_linear_bias_relative())
    row["differential_probability"] = float(sbox.maximal_difference_probability())
    row["boomerang_uniformity"] = int(sbox.boomerang_uniformity()) if sbox.is_permutation() else -1
    row["diff_branch"] = int(sbox.differential_branch_number())
    row["linear_branch"] = int(sbox.linear_branch_number())
    row["linearity"] = int(sbox.linearity())
    row["bic"] = sbox_bic(sbox) 
    row["sac"] = avg_sac(sbox) 
    return row 


for name, sbox in sboxes.items():
    if len(sbox) == 8:
        print("pickling:", name)
        df_dict[name] = create_row(sbox)

# Add the 8 sboxes
sb_8 = sboxes_8()
for name in sb_8.keys():
    sbox = SBox(*sb_8[name])
    print("pickling:", name)
    df_dict[name] = create_row(sbox)


df = pd.DataFrame.from_dict(df_dict).T
with open(pickle_filename, "wb") as f:     
    pickle.dump(df, f)

pickling: AES
pickling: ARIA_s2
pickling: Anubis
pickling: BelT
pickling: CLEFIA_S0
pickling: CLEFIA_S1
pickling: CMEA
pickling: CSA
pickling: CSS


/home/aolguin1/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/aolguin1/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


pickling: CS_cipher
pickling: Camellia
pickling: Chiasmus
pickling: Crypton_0_5
pickling: Crypton_1_0_S0
pickling: Crypton_1_0_S1
pickling: Crypton_1_0_S2
pickling: Crypton_1_0_S3
pickling: DBlock
pickling: E2
pickling: Enocoro
pickling: FLY
pickling: Fantomas
pickling: FlexAEAD
pickling: ForkSkinny_8
pickling: Fox
pickling: Iceberg
pickling: Iraqi
pickling: Kalyna_pi0
pickling: Kalyna_pi1
pickling: Kalyna_pi2
pickling: Kalyna_pi3
pickling: Khazad
pickling: Kuznechik
pickling: Kuznyechik
pickling: Lilliput_AE
pickling: MD2
pickling: Picaro
pickling: Remus_8
pickling: Romulus
pickling: SEED_S0
pickling: SEED_S1
pickling: SKINNY_8
pickling: SMS4
pickling: SNOW_3G_sq
pickling: Safer
pickling: Scream
pickling: Skipjack
pickling: Streebog
pickling: Stribog
pickling: Turing
pickling: Twofish_p0
pickling: Twofish_p1
pickling: Whirlpool
pickling: ZUC_S0
pickling: ZUC_S1
pickling: Zorro
pickling: iScream
pickling: newDES
pickling: S1
pickling: S2
pickling: S3
pickling: S4
pickling: S5
pickling:

In [43]:
df

,box,inverse,nonlinearity,linear_probability,differential_probability,boomerang_uniformity,diff_branch,linear_branch,linearity,bic,sac
AES,"[99, 124, 119, 123, 242, 107, 111, 197, 48, 1,...","[82, 9, 106, 213, 48, 54, 165, 56, 191, 64, 16...",112,0.0625,0.015625,6,2,2,32,0.134125,0.504883
ARIA_s2,"[226, 78, 84, 252, 148, 194, 74, 204, 98, 13, ...","[48, 104, 153, 27, 135, 185, 33, 120, 80, 57, ...",112,0.0625,0.015625,6,2,2,32,0.134125,0.50293
Anubis,"[167, 211, 230, 113, 208, 172, 77, 121, 58, 20...","[167, 211, 230, 113, 208, 172, 77, 121, 58, 20...",94,0.132812,0.03125,18,2,2,68,0.254902,0.500244
BelT,"[177, 148, 186, 200, 10, 8, 245, 59, 54, 109, ...","[10, 167, 28, 156, 17, 58, 144, 145, 5, 206, 4...",102,0.101562,0.03125,20,2,2,52,0.168409,0.499512
CLEFIA_S0,"[87, 73, 209, 198, 47, 51, 116, 251, 149, 109,...","[69, 187, 253, 144, 163, 135, 40, 226, 106, 94...",100,0.109375,0.039062,32,3,3,56,0.333333,0.539062
...,...,...,...,...,...,...,...,...,...,...,...
S4,"[0, 29, 165, 95, 22, 58, 2, 170, 124, 186, 166...","[0, 226, 6, 147, 232, 49, 149, 22, 63, 69, 164...",112,0.0625,0.015625,6,2,2,32,0.1261,0.5
S5,"[0, 29, 165, 95, 22, 58, 2, 170, 71, 176, 163,...","[0, 226, 6, 241, 138, 83, 247, 116, 63, 69, 16...",112,0.0625,0.015625,6,2,2,32,0.131696,0.499512
S6,"[27, 142, 54, 165, 210, 43, 173, 39, 83, 93, 2...","[127, 255, 118, 83, 141, 238, 90, 88, 233, 182...",112,0.0625,0.015625,6,2,2,32,0.1261,0.495361
S7,"[18, 248, 0, 161, 73, 221, 48, 98, 133, 69, 61...","[2, 130, 11, 81, 240, 147, 88, 90, 148, 180, 2...",112,0.0625,0.015625,6,2,2,32,0.1261,0.496094


In [10]:
type(df.T.to_dict()["AES"]["nonlinearity"])

int